In [4]:
# https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767
DEBUG = True

import re, random, math, csv, io, string, itertools
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
hp = dict(
    n_layers = 2,
    hidden_size = 512,
    fc_size = 512,
    dropout = 0.9,
    batch_size = 20,
    lr = 0.001,
    lr_decay = 0.9999,
    min_lr = 0.00001,
    grad_clip = 5.,
    cuda = False,
    num_epoch = 5,
    max_length = 10
)

In [36]:
class Voc:
    SOS = "!"
    EOS = "#"
    SOS_ID = 0
    EOS_ID = 1
    def __init__(self):
        self.word2index = {self.SOS:0, self.EOS:1}
        self.word2count = {}
        self.index2word = {0:self.SOS, 1:self.EOS}
        self.n_words = 2 # Count SOS and EOS

    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    
def string2indicies(voc, text):
    return [voc.word2index[c] for c in text]
    
def indicies2string(voc, indicies):
    return "".join([voc.index2word[i] for i in indicies])

voc = Voc()

for c in itertools.chain(range(ord('a'), ord('z')+1),range(ord('A'),ord('Z')+1),(ord(" "),)):
    voc.index_word(chr(c))
# print(f'vocabulary size: {voc.n_words}')

In [37]:
class PairGenerator:
#     vocabulary = [chr(i) for i in itertools.chain(range(ord('a'), ord('z')+1),range(ord('A'),ord('Z')+1))]
    word_len_interval = {"a":2,"b":7}
    sent_len_interval = {"a":1,"b":10}
    
    def __init__(self,voc):
        self.voc = voc
        self.vocabulary = [c for c in voc.word2index.keys() if c not in {voc.SOS, voc.EOS}]
        
    def gen_word_pair(self):
        word_len = int(random.uniform(**self.word_len_interval))
#         word = random.choices(self.vocabulary,k=word_len)
        word = np.random.choice(self.vocabulary,word_len)
        return "".join(word), "".join(list(reversed(word)))
    
    def gen_pair(self):
        num_words = int(random.uniform(**self.sent_len_interval))
        inp, out = zip(*[self.gen_word_pair() for _ in range(num_words)])
        return self.voc.SOS+" ".join(inp)+self.voc.EOS, self.voc.SOS+" ".join(out)+self.voc.EOS
    
    def gen_batch(self, n_unrollings):
        inp, out = zip(*[self.gen_pair() for _ in range(n_unrollings)])
        return inp, out
        
pg = PairGenerator(voc)

In [38]:
voc.index_words("!HKepmc#")

In [39]:
np.random.choice(pg.vocabulary,3)

array(['c', 'T', 'I'], dtype='<U1')

In [40]:
X,Y = pg.gen_batch(n_unrollings=10)

In [44]:
class GRU:
    hp = dict(
        state_sz=5,
        n_classes=2,
        input_dim=2,
        ckpt_path="./checkpoints/"
    )
    def __init__(self, **hyper_parameters):
        if hyper_parameters is not None:
            for k, v in hyper_parameters.items():
                self.hp[k] = v
        self.__graph__()
    
    def _init(self):
        x = tf.placeholder(tf.int32, [None, None], "x") #batch*voc
        y = tf.placeholder(tf.int32, [None, None], "y") #batch*voc
        emb = tf.placeholder(tf.int32, [None, None],)
        
        # batch_sz*seq_len -> batch_sz*seq_ln*voc_sz
        embs = tf.get_variable('emb', [num_classes, state_size]) 
        rnn_inputs = tf.nn.embedding_lookup(embs, x)
        
        #batch*state_sz
        init_state = tf.placeholder(tf.int32, [None, self.hp['state_sz']], "init_state")
    
    def _weights():
        self.wz = tf.get_variable(
            "w[z]", shape=[self.hp['state_sz'], self.hp['state_sz']],
            initializer=tf.contrib.layers.xavier_initializer()
        )
        self.uz = tf.get_variable(
            "u[z]", shape=[self.hp['state_sz'], self.hp['state_sz']],
            initializer=tf.contrib.layers.xavier_initializer()
        )
        self.bz = tf.get_variable(
            "b[z]", shape=[self.hp['state_sz']],
            initializer=tf.constant_initializer(0.)
        )
        
        self.wr = tf.get_variable(
            "w[r]",  shape=[self.hp['state_sz'], self.hp['state_sz']],
            initializer=tf.contrib.layers.xavier_initializer()
        )
        self.ur = tf.get_variable(
            "u[r]",  shape=[self.hp['state_sz'], self.hp['state_sz']],
            initializer=tf.contrib.layers.xavier_initializer()
        )
        self.br = tf.get_variable(
            "b[r]", shape=[self.hp['state_sz']],
            initializer=tf.constant_initializer(0.)
        )
        
        self.wh = tf.get_variable(
            "w[h]",  shape=[self.hp['state_sz'], self.hp['state_sz']],
            initializer=tf.contrib.layers.xavier_initializer()
        )
        self.uh = tf.get_variable(
            "w[h]",  shape=[self.hp['state_sz'], self.hp['state_sz']],
            initializer=tf.contrib.layers.xavier_initializer()
        )
        self.bh = tf.get_variable(
            "b[h]", shape=[self.hp['state_sz']],
            initializer=tf.constant_initializer(0.)
        )
        
        # layer to decode results of GRU 
        self.wo= tf.get_variable(
            'w[out]', shape=[self.hp['state_sz'], self.hp['n_classes']], 
            initializer=tf.contrib.layers.xavier_initializer()
        )
        self.bo = tf.get_variable(
            'b[out]', shape=[self.hp['num_classes']], 
            initializer=tf.constant_initializer(0.)
        )
    
    def __graph__(self):
        # time cycle step
        def step(prev_state, x):
            z = tf.matmul(x,self.wz) + tf.matmul(prev, self.uz) + self.bz
            r = tf.matmul(x,self.wr) + tf.matmul(prev, self.ur) + self.br
            h = tf.matmul(x,self.wh) + tf.matmul(h*prev, self.uh) + self.bh
            return (1-z)*prev + z*h
        
        tf.reset_default_graph()
        self.init()
        self.weights()
        states = tf.scan(
            step,
            #batch_sz*seq_ln*voc_sz -> seq_len*batch_sz*voc_sz
            tf.transpose(self.rnn_inputs,[1,0,2]),
            initializer=self.init_state
        )
        # seq_len*batch_sz*voc_sz -> batch_sz*seq_ln*voc_sz
        states = tf.transpose(states,[1,0,2])
        
        states_reshaped = tf.reshape(states, [-1, state_size])
        logits = tf.matmul(states_reshaped, self.wo) + self.bo
        
        self.last_state = states[-1]
        self.predictions = tf.nn.softmax(logits)
        self.loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(logits,self.y)
        )
        self.train_op = tf.train.AdagradOptimizer(learning_rate=0.2).minimize(self.loss)
        
    def train(self,x_tarin, y_train,n_epochs = 10):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            train_loss = 0
            try:
                for epoch_num in range(n_epochs):
                    pass
            except KeyboardInterrupt as ex:
                print("Interrupted by user at")

In [45]:
string2indicies(voc, X[0])

[0, 23, 3, 12, 10, 18, 1]

In [60]:
len_ls = list(map(len,X))

mx = max(len_ls)
diff = list(map(lambda x: mx - x, len_ls))

In [61]:
voc.word2index[' ']*xdiff

[41, 10, 12, 44, 0, 10, 32, 2, 44, 19]

In [48]:
indicies2string(voc, [0, 23, 3, 12, 10, 18, 1])

'!vbkiq#'